In [18]:
import pandas as pd
import numpy as np
import maboss
from create_generic_models.update_phenotypes_generic_models import generic_models_update_phenotypes


In [19]:
# chose folder where we want all the personalized boolean models and associated results saved 
drug_interest = 'Refametinib' #Pictilisib, 'Avagacestat' AZD8931 
tissue_interest = 'PAN_CANCER'
tissue_remove = 'Haematopoietic and Lymphoid'



In [ ]:
original_data_dir = 'models/generic_models/PAN_CANCER'

generic_model_bnd = f'{original_data_dir}/Montagud2022_PAN_Cancer.bnd'
generic_model_cfg = f'{original_data_dir}/Montagud2022_PAN_Cancer.cfg'

In [21]:
input_nodes = ['EGF', 'FGF', 'TGFB', 'Androgen', 'Hypoxia', 'Nutrients', 'Carcinogen', 'Acidosis', 'TNF', 'SPOP']
phenotypes_interest = [
        "Proliferation",
        "Invasion",
        "DNA_Repair",
        "Migration",
        "Apoptosis",
    ]

In [23]:
# update phenotypes 
generic_models_update_phenotypes(phenotypes_interest, original_data_dir, original_data_dir)

Updated SMAD.is_internal=1 in Montagud2022_PAN_Cancer.cfg
Updated PI3K.is_internal=1 in Montagud2022_PAN_Cancer.cfg
Updated MAP3K3.is_internal=1 in Montagud2022_PAN_Cancer.cfg
Updated MXI1.is_internal=1 in Montagud2022_PAN_Cancer.cfg
Updated Hypoxia.is_internal=1 in Montagud2022_PAN_Cancer.cfg
Updated CDH2.is_internal=1 in Montagud2022_PAN_Cancer.cfg
Updated CCNB1.is_internal=1 in Montagud2022_PAN_Cancer.cfg
Updated MYC.is_internal=1 in Montagud2022_PAN_Cancer.cfg
Updated PIP3.is_internal=1 in Montagud2022_PAN_Cancer.cfg
Updated PTEN.is_internal=1 in Montagud2022_PAN_Cancer.cfg
Updated ETS1.is_internal=1 in Montagud2022_PAN_Cancer.cfg
Updated NCOA3.is_internal=1 in Montagud2022_PAN_Cancer.cfg
Updated P21.is_internal=1 in Montagud2022_PAN_Cancer.cfg
Updated PTCH1.is_internal=1 in Montagud2022_PAN_Cancer.cfg
Updated VHL.is_internal=1 in Montagud2022_PAN_Cancer.cfg
Updated TWIST1.is_internal=1 in Montagud2022_PAN_Cancer.cfg
Updated EEF2K.is_internal=1 in Montagud2022_PAN_Cancer.cfg
Update

In [26]:
model_pers_lung = maboss.load(generic_model_bnd, generic_model_cfg)

results = pd.DataFrame(index=input_nodes, columns=phenotypes_interest)

threshold_diff = 0.01
initial_max_time = 5
max_time_limit = 30

for active_node in input_nodes:
    model_pers_lung.network.set_istate(active_node, [0, 1])
    for inactive_node in input_nodes:
        if inactive_node != active_node:
            model_pers_lung.network.set_istate(inactive_node, [1, 0])
    converged = False
    current_max_time = initial_max_time
    while not converged and current_max_time < max_time_limit:
        model_pers_lung.update_parameters(
            time_tick=0.1, max_time=current_max_time, sample_count=500
        )
        res_lung_pers = model_pers_lung.run()
        last_state = res_lung_pers.get_nodes_probtraj()
        if len(last_state) < 10:
            print(
                f"[{active_node}] Warning: Not enough data for convergence check."
            )
            break
        diff = last_state.diff().abs()
        max_change_recent = diff.tail(10).max()
        # print(f"Max change over last 10 time steps for {active_node}:\n{max_change_recent}\n")
        if (max_change_recent > threshold_diff).any():
            current_max_time += 1
        else:
            converged = True
    if not converged:
        print(
            f"[{active_node}] Did not fully converge by max_time = {max_time_limit}."
        )
    final_probs = last_state.iloc[-1]
    # print(f"[{active_node}] Final probabilities:\n{final_probs}\n")
    for phenotype in phenotypes_interest:
        if phenotype in final_probs.index:
            results.loc[active_node, phenotype] = final_probs[phenotype]
    
    results.loc['Overal Mean'] = results.mean()

print(results)
results.to_csv('results/Refametinib_PAN_CANCER/generic_results/phenotype_distribution_generic.csv')

            Proliferation  Invasion DNA_Repair Migration Apoptosis
EGF              0.232001     0.158      0.002  0.119179  0.130676
FGF              0.254719  0.124128      0.002     0.034  0.317466
TGFB             0.146566  0.724872        0.0  0.150947  0.410843
Androgen         0.226001     0.158      0.002  0.117179  0.132676
Hypoxia          0.049131  0.091545        0.0  0.044081  0.090078
Nutrients        0.383216  0.119624        0.0  0.062559   0.09193
Carcinogen       0.168661     0.099   0.386754     0.012  0.695798
Acidosis          0.21385     0.118      0.002  0.054945  0.125739
TNF              0.089473  0.940997        0.0  0.340599     0.052
SPOP             0.372392  0.303326      0.002  0.118849  0.069099
Overal Mean      0.212118  0.282817   0.040106  0.105034  0.213279
